In [2]:
from pulp import *
import pandas as pd

### Problem 1

In [4]:
#Define the LP problem
prob1 = LpProblem("Problem1", LpMinimize)
rc = ['West','Midwest','East']
nc = ['Ohio','NewJersey']
months = ['Sept','Oct']
tran_cost = 10
n_inv_cost = 5
r_inv_cost = 10

#read data from excel file
df_fx_trans = pd.read_excel('Problem1_tables.xlsx',sheet_name='Table1a')
df_demand = pd.read_excel('Problem1_tables.xlsx',sheet_name='Table1b')
df_inventory = pd.read_excel('Problem1_tables.xlsx',sheet_name='Table1c')
n_fixed_tran_cost = df_fx_trans.iloc[2:4,2].values.tolist()
r_fixed_tran_cost = df_fx_trans.iloc[4:,2].values.tolist()
sept_demand = df_demand.iloc[[2, 4, 6], 2].values.tolist()
oct_demand = df_demand.iloc[[3, 5, 7], 2].values.tolist()
n_initial_inv = df_inventory.iloc[2:4,2].values.tolist()
r_initial_inv = df_inventory.iloc[4:,2].values.tolist()

#Set up parameters
nftc = dict(zip(nc,n_fixed_tran_cost))

ohio_ftc = r_fixed_tran_cost[0:2]
nj_ftc = r_fixed_tran_cost[2:4]
o_rftc = dict(zip(['West', 'Midwest'], ohio_ftc))
nj_rftc = dict(zip(['Midwest', 'East'], nj_ftc))

sd = dict(zip(rc,sept_demand))
od = dict(zip(rc,oct_demand))
ni = dict(zip(nc,n_initial_inv))
ri = dict(zip(rc,r_initial_inv))


In [5]:
# Define decision variables
n_ship = LpVariable.dicts("nc", [(m,n) for m in months for n in nc], lowBound=0, cat='Continuous')
r_ship = LpVariable.dicts("rc", [(m, n, r) for m in months for n in nc for r in rc], lowBound=0, cat='Continuous')
n_vars = LpVariable.dicts("nh_to_nc", [(m, n) for m in months for n in nc], cat='Binary')
r_vars = LpVariable.dicts("nc_to_rc", [(m, n, r) for m in months for n in nc for r in rc], cat='Binary')
n_inv = LpVariable.dicts("n_inv", [(m, n) for m in months for n in nc], lowBound=0, cat='Continuous')
r_inv = LpVariable.dicts("r_inv", [(m, r) for m in months for r in rc], lowBound=0, cat='Continuous')

In [6]:
# Objective Function
prob1 += lpSum([tran_cost * n_ship[(m, n)] for m in months for n in nc] +
    [tran_cost * r_ship[(m, n, r)] for m in months for n in nc for r in rc] +
    [n_inv_cost * n_inv[(m, n)] for m in months for n in nc] +
    [r_inv_cost * r_inv[(m, r)] for m in months for r in rc] +
    [nftc[n] * n_vars[(m, n)] for m in months for n in nc] +
    [o_rftc[r] * r_vars[(m, 'Ohio', r)] for m in months for r in o_rftc] +
    [nj_rftc[r] * r_vars[(m, 'NewJersey', r)] for m in months for r in nj_rftc]), "Total_Cost"

# Production Capacity Constraint
for m in months:
    for n in nc:
        prob1 += lpSum(n_ship[(m, n)] for n in nc) <= 50000 #* n_vars[(m, n)]

# Setting Binary Contraints
A = 100000
for m in months:
    for n in nc:
        prob1 += n_ship[(m, n)] <= n_vars[(m, n)] * A
        for r in rc:
            prob1 += r_ship[(m, n, r)] <= r_vars[(m, n, r)] * A

# Transportation Constraint
for m in months:
    prob1 += r_ship[(m,'Ohio','East')] == 0
    prob1 += r_ship[(m,'NewJersey','West')] == 0
    prob1 += r_vars[(m,'Ohio','East')] == 0
    prob1 += r_vars[(m,'NewJersey','West')] == 0

# Shipping Capacity Constraint
for m in months:
    for n in nc:
        if m == 'Sept':
            prob1 += ni[n] + n_ship[(m, n)] >= lpSum(r_ship[(m, n, r)] for r in rc)
        else:
            prob1 += n_inv[('Sept', n)] + n_ship[(m, n)] >= lpSum(r_ship[(m, n, r)] for r in rc)

for m in months:
    if m == 'Sept':
        for r in rc:
            prob1 += lpSum([r_ship[(m, n, r)] for n in nc]) + ri[r] >= sd[r]
    else:
        for r in rc:
            prob1 += lpSum([r_ship[(m, n, r)] for n in nc]) + r_inv[('Sept', r)] >= od[r]
            
# Inventory Constraint
for m in months:
    if m == 'Sept':
        for n in nc:
            prob1 += n_inv[(m, n)] == ni[n] + n_ship[(m, n)] - lpSum([r_ship[(m, n, r)] for r in rc])
        for r in rc:
            prob1 += r_inv[(m, r)] == ri[r] + lpSum([r_ship[(m, n, r)] for n in nc]) - sd[r]
    else:
        for n in nc:
            prob1 += n_inv[(m, n)] == n_inv[('Sept', n)] + n_ship[(m, n)] - lpSum([r_ship[(m, n, r)] for r in rc])
        for r in rc:
            prob1 += r_inv[(m, r)] == r_inv[('Sept', r)] + lpSum([r_ship[(m, n, r)] for n in nc]) - od[r]

In [7]:
prob1.solve()
print("Status:",LpStatus[prob1.status])

Status: Optimal


In [8]:
for v in prob1.variables():
    print(v.name, "=", v.varValue)
print("Total Cost=", value(prob1.objective))

n_inv_('Oct',_'NewJersey') = 0.0
n_inv_('Oct',_'Ohio') = 0.0
n_inv_('Sept',_'NewJersey') = 5000.0
n_inv_('Sept',_'Ohio') = 20000.0
nc_('Oct',_'NewJersey') = 50000.0
nc_('Oct',_'Ohio') = 0.0
nc_('Sept',_'NewJersey') = 18000.0
nc_('Sept',_'Ohio') = 32000.0
nc_to_rc_('Oct',_'NewJersey',_'East') = 1.0
nc_to_rc_('Oct',_'NewJersey',_'Midwest') = 1.0
nc_to_rc_('Oct',_'NewJersey',_'West') = 0.0
nc_to_rc_('Oct',_'Ohio',_'East') = 0.0
nc_to_rc_('Oct',_'Ohio',_'Midwest') = 0.0
nc_to_rc_('Oct',_'Ohio',_'West') = 1.0
nc_to_rc_('Sept',_'NewJersey',_'East') = 1.0
nc_to_rc_('Sept',_'NewJersey',_'Midwest') = 0.0
nc_to_rc_('Sept',_'NewJersey',_'West') = 0.0
nc_to_rc_('Sept',_'Ohio',_'East') = 0.0
nc_to_rc_('Sept',_'Ohio',_'Midwest') = 1.0
nc_to_rc_('Sept',_'Ohio',_'West') = 1.0
nh_to_nc_('Oct',_'NewJersey') = 1.0
nh_to_nc_('Oct',_'Ohio') = 0.0
nh_to_nc_('Sept',_'NewJersey') = 1.0
nh_to_nc_('Sept',_'Ohio') = 1.0
r_inv_('Oct',_'East') = 0.0
r_inv_('Oct',_'Midwest') = 0.0
r_inv_('Oct',_'West') = 0.0
r_inv_

### The Optimal Cost is 2460000.0